## Logistic Regression Model for Kepler Space Telescope Data
The purpose of this project is to reinforce my data science and machine learning knowledge, and to help others in their journeys. I'm not an expert on these topics, and I'm conscious that there are a lot of tools, concepts and techniques that I need to master. That's why I develop this project, to share with the community the tools, concepts and techniques, that I have learned across my data science journey. I'm open to comments, critics and feedback that would help me to develop learn best practices and corrections in case I was wrong.

#### About Logistic Regression
- It is a classification model.
- Mostly used to predict a discrete set of categories, such as Yes/No, Young/Old, Cold/Hot, etc.
- They are stronger with linear relationships.
- They work only with numbers (All your data need to be numerical).
- By default, logistic regression cannot be used for classification tasks that have more than two targets.
- Since we have more than 2 classes to predict, we are going to use multinomial multi_class.

### Importing basic modules for data extraction and analysis

In [2]:
#import warnings
#warnings.simplefilter('ignore')
import pandas as pd
import numpy as np

In [3]:
kepler_df = pd.read_csv('../../data/exoplanet_data.csv')
kepler_df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
kepler_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

In [5]:
print(f"The dataframe has a length of {len(kepler_df.columns)} columns.")
print(f"There are 3 outcomes/predictions/targets: {set(kepler_df['koi_disposition'])}")

The dataframe has a length of 41 columns.
There are 3 outcomes/predictions/targets: {'CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'}


#### Replacing categorical target values with numerical values.

In [6]:
kepler_df['koi_disposition'] = kepler_df['koi_disposition'].replace({'CONFIRMED': 0, 'FALSE POSITIVE': 1, 'CANDIDATE': 2})

### Taking a look to the relationship between features and target

#### Relationship between the target which is 'koi_disposition' and all the features available in the dataset.

In [7]:
correlation = kepler_df.corr()
correlation['koi_disposition']

koi_disposition      1.000000
koi_fpflag_nt        0.000416
koi_fpflag_ss        0.013503
koi_fpflag_co        0.008531
koi_fpflag_ec        0.008041
koi_period           0.124647
koi_period_err1      0.099048
koi_period_err2     -0.099048
koi_time0bk          0.070445
koi_time0bk_err1     0.147719
koi_time0bk_err2    -0.147719
koi_impact           0.010607
koi_impact_err1      0.058572
koi_impact_err2     -0.013980
koi_duration         0.029554
koi_duration_err1    0.156587
koi_duration_err2   -0.156587
koi_depth            0.008694
koi_depth_err1       0.001797
koi_depth_err2      -0.001797
koi_prad             0.001485
koi_prad_err1        0.003135
koi_prad_err2       -0.000998
koi_teq              0.021275
koi_insol            0.012070
koi_insol_err1       0.014604
koi_insol_err2      -0.014159
koi_model_snr       -0.016351
koi_tce_plnt_num    -0.095550
koi_steff            0.071048
koi_steff_err1       0.173227
koi_steff_err2      -0.148902
koi_slogg           -0.071437
koi_slogg_

### Plotting heatmaps

#### Two ways of creating a heatmap with Plotly.

In [13]:
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as figfac
from plotly.offline import offline, iplot

fig = go.Figure(data=go.Heatmap(z = correlation.iloc[:11, :11].round(3).values.tolist(),
                                x=correlation.iloc[:11, :11].columns.to_list(),
                                y = correlation.iloc[:11, :11].index.to_list(),
                                colorscale= 'icefire',
                                text=correlation.iloc[:11, :11].round(4).values,
                                texttemplate="%{text}",
                                textfont={"size":12}))

fig.update_layout(title_text='First 10 Features Heatmap Correlation with the Target', title_x=0.5)

fig.show()
fig.write_image("logistic_regression_complete_heatmap.png")
offline.plot(fig, filename='logistic_regression_complete_heatmap.html')

'logistic_regression_complete_heatmap.html'

In [14]:
iplot(figfac.create_annotated_heatmap(correlation.iloc[:11, :11].round(3).values, x=correlation.iloc[:11, :11].columns.to_list(), 
                                  y=correlation.iloc[:11, :11].index.to_list(), annotation_text=correlation.iloc[:11, :11].round(4).values, colorscale= 'bluered'))



### Setting target and features

#### Splitting up my target columns from the rest of my features.

In the following cell, it will be set the target (y) and the features (X), in this case for the target, the selected column will be 'koi_disposition', meanwhile for the features variable, the rest of the columns will be selected by droping the 'koi_disposition' column.

In [47]:
features = kepler_df.drop('koi_disposition', axis = 1)
target = kepler_df['koi_disposition']

print(features.shape)
print(target.shape)

(6991, 40)
(6991,)


### Initiating the Logistic Regression model

Importing LogisticRegression and creating a LogisticRegression with a parameter of max iterations of 1000.

In [67]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=1000)


#### Import the train_test_split, and split the target and features into train and test.

The parameters used in this project were:
- features (X)
- target (y)
- test_size: it represents the proportion of the actual data, that will be considered as test; it should be between 0 and 1, and the default value is 0.25, in this case the test size is 0.30.
- random_state: it is a parameter that helps you get the same results, for instance, suppose you want to get a random number between 1 and 1000, and you get 50. It is very unlikely that in your next run, you get 50 again, that's what the 'random_state' is basically for; by setting the same 'random_state' on both runs, you will get the same number of times you run the same parameter for 'random_state'.

In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

### Preprocessing

#### Scaling our data

In [90]:
from sklearn.preprocessing import MinMaxScaler


X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)


#### Fitting the LogisticRegression model with the scaled X_train

In [70]:
logreg.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

#### Selecting the features with RFE (Recursive Feature Elimination)

In [71]:
from sklearn.feature_selection import RFE
rfe = RFE(logreg, n_features_to_select=10, step=1)
features_selected = rfe.fit(X_train_scaled, y_train)

In [72]:
relevant_features = features.loc[:,features_selected.support_]
relevant_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_time0bk_err1,koi_time0bk_err2,koi_duration_err1,koi_duration_err2,koi_model_snr,koi_steff_err1
0,0,0,0,0,0.003520,-0.003520,0.11600,-0.11600,25.8,81
1,0,1,0,0,0.000581,-0.000581,0.03410,-0.03410,76.3,158
2,0,1,0,0,0.000115,-0.000115,0.00537,-0.00537,505.6,157
3,0,0,0,0,0.001130,-0.001130,0.04200,-0.04200,40.9,169
4,0,0,0,0,0.001900,-0.001900,0.06730,-0.06730,40.2,189


In [73]:
relevant_corr_df = pd.merge(target, relevant_features, left_index=True, right_index=True)
relevant_corr_df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_time0bk_err1,koi_time0bk_err2,koi_duration_err1,koi_duration_err2,koi_model_snr,koi_steff_err1
0,0,0,0,0,0,0.003520,-0.003520,0.11600,-0.11600,25.8,81
1,1,0,1,0,0,0.000581,-0.000581,0.03410,-0.03410,76.3,158
2,1,0,1,0,0,0.000115,-0.000115,0.00537,-0.00537,505.6,157
3,0,0,0,0,0,0.001130,-0.001130,0.04200,-0.04200,40.9,169
4,0,0,0,0,0,0.001900,-0.001900,0.06730,-0.06730,40.2,189


In [74]:
relevant_corr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   int64  
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_time0bk_err1   6991 non-null   float64
 6   koi_time0bk_err2   6991 non-null   float64
 7   koi_duration_err1  6991 non-null   float64
 8   koi_duration_err2  6991 non-null   float64
 9   koi_model_snr      6991 non-null   float64
 10  koi_steff_err1     6991 non-null   int64  
dtypes: float64(5), int64(6)
memory usage: 600.9 KB


### Taking a look to the relationship between the top 10 most relevant features and target

In [75]:
rel_correlation = relevant_corr_df.corr()
rel_correlation['koi_disposition']

koi_disposition      1.000000
koi_fpflag_nt        0.000416
koi_fpflag_ss        0.013503
koi_fpflag_co        0.008531
koi_fpflag_ec        0.008041
koi_time0bk_err1     0.147719
koi_time0bk_err2    -0.147719
koi_duration_err1    0.156587
koi_duration_err2   -0.156587
koi_model_snr       -0.016351
koi_steff_err1       0.173227
Name: koi_disposition, dtype: float64

In [91]:
fig = go.Figure(data=go.Heatmap(z = rel_correlation.round(3).values.tolist(),
                                x = rel_correlation.columns.to_list(),
                                y = rel_correlation.index.to_list(),
                                colorscale = 'ylgnbu',
                                text = rel_correlation.round(4).values,
                                texttemplate = "%{text}",
                                textfont = {"size":12}))

fig.update_layout(title_text='Top 10 Most Relevant Features Heatmap Correlation with the Target', title_x=0.5)

fig.show()

In [ ]:
iplot(figfac.create_annotated_heatmap(rel_correlation.round(3).values, x=rel_correlation.columns.to_list(), 
                                  y=rel_correlation.index.to_list(), annotation_text=rel_correlation.round(4).values, colorscale= 'ylgnbu'))

### Scaling in a pipeline

In [78]:
steps = [('scaler', MinMaxScaler()),
         ('logistic_regression', logreg)]

In [79]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps)

### Hyperparameter tuning

In [80]:
c_space = np.logspace(-4, 4, 20)
parameters = {'logistic_regression__C':c_space,
              'logistic_regression__penalty':['l2', 'none'],
              'logistic_regression__solver': ['lbfgs','newton-cg','sag'],
             'logistic_regression__multi_class':['multinomial']}

In [ ]:
from sklearn.model_selection import GridSearchCV
logreg_cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 3)

### Running train_test_split but only with the relevant_features, selected by the RFE

In [81]:
X_train, X_test, y_train, y_test = train_test_split(relevant_features, target, test_size=0.3, random_state=42)

### Fitting our model with all the possible parameters options

In [83]:
logreg_cv.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END logistic_regression__C=0.0001, logistic_regression__max_iter=1000, logistic_regression__multi_class=multinomial, logistic_regression__penalty=l2, logistic_regression__solver=lbfgs;, score=0.508 total time=   0.0s
[CV 2/5] END logistic_regression__C=0.0001, logistic_regression__max_iter=1000, logistic_regression__multi_class=multinomial, logistic_regression__penalty=l2, logistic_regression__solver=lbfgs;, score=0.508 total time=   0.0s
[CV 3/5] END logistic_regression__C=0.0001, logistic_regression__max_iter=1000, logistic_regression__multi_class=multinomial, logistic_regression__penalty=l2, logistic_regression__solver=lbfgs;, score=0.507 total time=   0.0s
[CV 4/5] END logistic_regression__C=0.0001, logistic_regression__max_iter=1000, logistic_regression__multi_class=multinomial, logistic_regression__penalty=l2, logistic_regression__solver=lbfgs;, score=0.508 total time=   0.0s
[CV 5/5] END logistic_regression_

c:\users\raulm\anaconda3\envs\pythondata\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters



[CV 1/5] END logistic_regression__C=0.0001, logistic_regression__max_iter=1000, logistic_regression__multi_class=multinomial, logistic_regression__penalty=none, logistic_regression__solver=lbfgs;, score=0.876 total time=   0.2s


c:\users\raulm\anaconda3\envs\pythondata\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters



[CV 2/5] END logistic_regression__C=0.0001, logistic_regression__max_iter=1000, logistic_regression__multi_class=multinomial, logistic_regression__penalty=none, logistic_regression__solver=lbfgs;, score=0.854 total time=   0.2s


...

c:\users\raulm\anaconda3\envs\pythondata\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters



[CV 4/5] END logistic_regression__C=10000.0, logistic_regression__max_iter=1000, logistic_regression__multi_class=multinomial, logistic_regression__penalty=none, logistic_regression__solver=sag;, score=0.845 total time=   0.2s


c:\users\raulm\anaconda3\envs\pythondata\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters



[CV 5/5] END logistic_regression__C=10000.0, logistic_regression__max_iter=1000, logistic_regression__multi_class=multinomial, logistic_regression__penalty=none, logistic_regression__solver=sag;, score=0.851 total time=   0.2s


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('logistic_regression',
                                        LogisticRegression(max_iter=1000))]),
             param_grid={'logistic_regression__C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'logistic_regression__max_iter': [1000],
                         'logistic_regression__multi_class': ['multinomial'],
                         'logistic_regression__penalty': ['l2', 'none'],
                         'logistic_regression__solver': ['lbfgs', 'newton-cg',
                                                         'sag']},
             verbose=3

### Collecting the predictions for the X_test

In [84]:
predict = logreg_cv.predict(X_test)

### Model metrics

In [85]:
from sklearn.metrics import classification_report, confusion_matrix
print("=======================================")
print("Models Accuracy")
print("=======================================")
print("Accuracy: {}%".format((logreg_cv.score(X_test, y_test))*100))
print("---------------------------------------")
print("Classification Report")
print("=======================================")
print(classification_report(y_test, predict))
print("---------------------------------------")
print("Best Parameters")
print("=======================================")
print("Tuned Model Parameters: {}".format(logreg_cv.best_params_))
print("---------------------------------------")
print("Confusion Matrix")
print("=======================================")
print(confusion_matrix(y_test, predict))
print("---------------------------------------")

Models Accuracy
Accuracy: 85.55767397521448%
---------------------------------------
Classification Report
              precision    recall  f1-score   support

           0       0.72      0.80      0.76       574
           1       0.98      1.00      0.99      1020
           2       0.76      0.62      0.68       504

    accuracy                           0.86      2098
   macro avg       0.82      0.81      0.81      2098
weighted avg       0.86      0.86      0.85      2098

---------------------------------------
Best Parameters
Tuned Model Parameters: {'logistic_regression__C': 78.47599703514607, 'logistic_regression__max_iter': 1000, 'logistic_regression__multi_class': 'multinomial', 'logistic_regression__penalty': 'l2', 'logistic_regression__solver': 'lbfgs'}
---------------------------------------
Confusion Matrix
[[ 462   12  100]
 [   0 1018    2]
 [ 182    7  315]]
---------------------------------------


### Prediction vs Actual DataFrame

In [43]:
predictor_df = pd.DataFrame({'Prediction':predict, 'Actual':y_test}).reset_index(drop=True)
predictor_df[['Prediction', 'Actual']] = predictor_df[['Prediction', 'Actual']].replace({0: 'CONFIRMED', 1: 'FALSE POSITIVE', 2 : 'CANDIDATE'})
predictor_df

,Prediction,Actual
0,FALSE POSITIVE,FALSE POSITIVE
1,CANDIDATE,CANDIDATE
2,FALSE POSITIVE,FALSE POSITIVE
3,FALSE POSITIVE,FALSE POSITIVE
4,FALSE POSITIVE,FALSE POSITIVE
...,...,...
2093,FALSE POSITIVE,FALSE POSITIVE
2094,CANDIDATE,CONFIRMED
2095,CONFIRMED,CANDIDATE
2096,CONFIRMED,CONFIRMED
